In [25]:
from fastai.vision.all import *
from fastaudio.all import *

In [28]:
path = Path('../pitch_accent')
path.ls()

(#6) [Path('../pitch_accent/all_labeles.csv'),Path('../pitch_accent/dict2_labels.csv'),Path('../pitch_accent/dict1_labels.csv'),Path('../pitch_accent/accentAudio'),Path('../pitch_accent/OjadMedia'),Path('../pitch_accent/.ipynb_checkpoints')]

In [41]:
data = pd.read_csv(path / 'all_labeles.csv')
data["is_valid"] = True
data.loc[data.type != 'ojad male',"is_valid"] = False
data

,path,pattern,kana,morae,drop,type,is_valid
0,accentAudio/ある.yomi000142BB_0596.mp3,頭高,アル,2,1,nhk,False
1,accentAudio/思う.yomi0006C617_043A.mp3,中高,オモウ,3,2,nhk,False
2,accentAudio/など.yomi000240B7_0028.mp3,頭高,ナド,2,1,nhk,False
3,accentAudio/私.yomi00092F63_0072.mp3,平板,ワタくシ,4,0,nhk,False
4,accentAudio/見る.yomi000A41BD_001E.mp3,頭高,ミル,2,1,nhk,False
...,...,...,...,...,...,...,...
163962,OjadMedia/立て-377_10_1_female.mp3,頭高,たて,2,1,ojad female,False
163963,OjadMedia/立てる-377_11_1_male.mp3,中高,たてる,3,2,ojad male,True
163964,OjadMedia/立てる-377_11_1_female.mp3,中高,たてる,3,2,ojad female,False
163965,OjadMedia/立とう-377_12_1_male.mp3,中高,たとう,3,2,ojad male,True


In [42]:
data.pattern

0         頭高
1         中高
2         頭高
3         平板
4         頭高
          ..
163962    頭高
163963    中高
163964    中高
163965    中高
163966    中高
Name: pattern, Length: 163967, dtype: object

In [22]:
cat = Categorize()
cat.setup(data.pattern)
cat.vocab

['None', '中高', '尾高', '平板', '頭高']

In [68]:
splits = ColSplitter()(data)
aud2spec = AudioToSpec.from_cfg(AudioConfig.Voice())

In [76]:
x_tfms = [lambda x: x[0], lambda x1: path/x1, AudioTensor.create, Resample(16_000), RemoveSilence(), ResizeSignal(2000)]
y_tfms = [lambda y: y["pattern"], cat]
dsets = Datasets(data, [x_tfms, y_tfms], splits = splits)
dsets

(#163967) [(AudioTensor([[0., 0., 0.,  ..., 0., 0., 0.]]), TensorCategory(4)),(AudioTensor([[0., 0., 0.,  ..., 0., 0., 0.]]), TensorCategory(1)),(AudioTensor([[0., 0., 0.,  ..., 0., 0., 0.]]), TensorCategory(4)),(AudioTensor([[0., 0., 0.,  ..., 0., 0., 0.]]), TensorCategory(3)),(AudioTensor([[0., 0., 0.,  ..., 0., 0., 0.]]), TensorCategory(4)),(AudioTensor([[0., 0., 0.,  ..., 0., 0., 0.]]), TensorCategory(4)),(AudioTensor([[0., 0., 0.,  ..., 0., 0., 0.]]), TensorCategory(3)),(AudioTensor([[0., 0., 0.,  ..., 0., 0., 0.]]), TensorCategory(3)),(AudioTensor([[0., 0., 0.,  ..., 0., 0., 0.]]), TensorCategory(3)),(AudioTensor([[0., 0., 0.,  ..., 0., 0., 0.]]), TensorCategory(4))...]

In [74]:
dsets[0][0].shape

torch.Size([1, 128, 251])

In [77]:
dls = dsets.dataloaders(after_batch=[aud2spec])
dls.one_batch()

(AudioSpectrogram([[[[-100., -100., -100.,  ..., -100., -100., -100.],
           [-100., -100., -100.,  ..., -100., -100., -100.],
           [-100., -100., -100.,  ..., -100., -100., -100.],
           ...,
           [-100., -100., -100.,  ..., -100., -100., -100.],
           [-100., -100., -100.,  ..., -100., -100., -100.],
           [-100., -100., -100.,  ..., -100., -100., -100.]]],
 
 
         [[[-100., -100., -100.,  ..., -100., -100., -100.],
           [-100., -100., -100.,  ..., -100., -100., -100.],
           [-100., -100., -100.,  ..., -100., -100., -100.],
           ...,
           [-100., -100., -100.,  ..., -100., -100., -100.],
           [-100., -100., -100.,  ..., -100., -100., -100.],
           [-100., -100., -100.,  ..., -100., -100., -100.]]],
 
 
         [[[-100., -100., -100.,  ..., -100., -100., -100.],
           [-100., -100., -100.,  ..., -100., -100., -100.],
           [-100., -100., -100.,  ..., -100., -100., -100.],
           ...,
           [-10

In [64]:
OpenAudio??

Init signature: OpenAudio(self, items)
Source:        
class OpenAudio(Transform):
    """
    Transform that creates AudioTensors from a list of files.
    """

    def __init__(self, items):
        self.items = items

    def encodes(self, i):
        o = self.items[i]
        return AudioTensor.create(o)

    def decodes(self, i) -> Path:
        return self.items[i]
File:           ~/.conda/envs/default/lib/python3.9/site-packages/fastaudio/core/signal.py
Type:           _TfmMeta
Subclasses:     


In [65]:
AudioBlock??

Init signature:
AudioBlock(
    cache_folder=None,
    sample_rate=16000,
    force_mono=True,
    crop_signal_to=None,
)
Source:        
class AudioBlock(TransformBlock):
    "A `TransformBlock` for audios"

    @delegates(audio_item_tfms)
    def __init__(self, cache_folder=None, **kwargs):
        item_tfms = audio_item_tfms(**kwargs)
        type_tfm = partial(AudioTensor.create, cache_folder=cache_folder)
        return super().__init__(
            type_tfms=type_tfm, item_tfms=item_tfms, batch_tfms=IntToFloatTensor
        )

    @classmethod
    @delegates(audio_item_tfms, keep=True)
    def from_folder(cls, path, **kwargs):
        "Build a `AudioBlock` from a `path` and caches some intermediary results"
        cache_folder = preprocess_audio_folder(path, **kwargs)
        return cls(cache_folder, **kwargs)
File:           ~/.conda/envs/default/lib/python3.9/site-packages/fastaudio/core/config.py
Type:           type
Subclasses:     


In [66]:
preprocess_audio_folder??

Signature:
preprocess_audio_folder(
    path,
    folders=None,
    output_dir=None,
    sample_rate=16000,
    force_mono=True,
    crop_signal_to=None,
    **kwargs,
)
Source:   
@delegates(PreprocessAudio, keep=True)
def preprocess_audio_folder(path, folders=None, output_dir=None, **kwargs):
    "Preprocess audio files in `path` in parallel using `n_workers`"
    path = Path(path)
    fnames = get_audio_files(path, recurse=True, folders=folders)
    output_dir = Path(ifnone(output_dir, path.parent / f"{path.name}_cached"))
    output_dir.mkdir(exist_ok=True)

    pp = PreprocessAudio(**kwargs)

    for i, fil in enumerate(fnames):
        out = output_dir / fnames[i].relative_to(path)
        aud = pp(fil)
        save_audio(str(out), aud, aud.sr)
    return output_dir
File:      ~/.conda/envs/default/lib/python3.9/site-packages/fastaudio/core/config.py
Type:      function


In [67]:
PreprocessAudio??

Init signature: PreprocessAudio(sample_rate=16000, force_mono=True, crop_signal_to=None)
Source:        
class PreprocessAudio:
    """
    Creates an audio tensor and run the basic preprocessing transforms on it.
    Used while preprocessing the audios, this is not a `Transform`.
    """

    @delegates(audio_item_tfms)
    def __init__(self, **kwargs):
        self.tfms = Pipeline(audio_item_tfms(**kwargs))

    def __call__(self, x):
        audio = AudioTensor.create(x)
        return self.tfms(audio)
File:           ~/.conda/envs/default/lib/python3.9/site-packages/fastaudio/core/config.py
Type:           type
Subclasses:     


In [72]:
audio_item_tfms??

Signature: audio_item_tfms(sample_rate=16000, force_mono=True, crop_signal_to=None)
Source:   
def audio_item_tfms(sample_rate=16000, force_mono=True, crop_signal_to=None):
    """
    Basic audio preprocessing transforms.
    """
    tfms = []
    if sample_rate is not None:
        tfms.append(Resample(sample_rate))
    if force_mono:
        tfms.append(DownmixMono())
    if crop_signal_to is not None:
        tfms.append(ResizeSignal(duration=crop_signal_to))
    return tfms
File:      ~/.conda/envs/default/lib/python3.9/site-packages/fastaudio/core/config.py
Type:      function
